In [1]:
#pip install fastparquet
#pip install pyarrrow

In [2]:
import pyarrow as pa
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
import fastparquet

In [3]:
# ler todos os arquivos csv do diretório e guardar em um objeto
path = '/home/vinicius.vancellote/data/parquet/'
all_files = list(filter(lambda x: '.parquet' in x, os.listdir(path)))
print (all_files)

['G1-2019-08-27.parquet', 'G1-2019-12-26.parquet', 'G1-2019-12-21.parquet', 'G1-2019-12-02.parquet', 'G1-2019-08-12.parquet', 'G1-2019-08-08.parquet', 'G1-2019-08-18.parquet', 'G1-2019-08-31.parquet', 'G1-2019-12-04.parquet', 'G1-2019-08-10.parquet', 'G1-2019-12-20.parquet', 'G1-2019-08-01.parquet', 'G1-2019-12-10.parquet', 'G1-2019-12-28.parquet', 'G1-2019-12-19.parquet', 'G1-2019-12-03.parquet', 'G1-2019-12-09.parquet', 'G1-2019-12-08.parquet', 'G1-2019-08-03.parquet', 'G1-2019-08-23.parquet', 'G1-2019-08-06.parquet', 'G1-2019-12-15.parquet', 'G1-2019-12-11.parquet', 'G1-2019-12-22.parquet', 'G1-2019-08-28.parquet', 'G1-2019-08-11.parquet', 'G1-2019-08-09.parquet', 'G1-2019-12-16.parquet', 'G1-2019-08-20.parquet', 'G1-2019-08-05.parquet', 'G1-2019-12-24.parquet', 'G1-2019-08-25.parquet', 'G1-2019-12-25.parquet', 'G1-2019-08-26.parquet', 'G1-2019-12-13.parquet', 'G1-2019-08-22.parquet', 'G1-2019-08-07.parquet', 'G1-2019-08-16.parquet', 'G1-2019-08-30.parquet', 'G1-2019-08-15.parquet',

In [4]:
dataset_analise = []

In [5]:
import json
from pandas import json_normalize
import geopandas as gpd
import xml.etree.cElementTree as etree

/home/vinicius.vancellote/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [6]:
# Abrindo o Arquivo GeoJson com o limite das Zonas Pluviometricas
zonasgpd = gpd.read_file('/home/vinicius.vancellote/Codigos_Python/Bases/EstacoesPluviometricas/Zonas_Pluviometricas.geojson')

In [7]:
zonasgpd = zonasgpd.drop(columns=['Endereço','Est','ShapeSTArea','ShapeSTLength','Cod'])

In [8]:
chuva = pd.read_csv("/home/vinicius.vancellote/data/json/chuva.csv")
chuva = chuva.drop(columns=['Unnamed: 0'])
chuva['data_particao'] = pd.to_datetime(chuva['data_particao'],format = '%Y-%m-%d' )
chuva['horario'] = pd.to_datetime(chuva['horario'],format = '%H:%M:%S').dt.time
chuva['DATE'] = chuva.apply(lambda row: pd.datetime.combine(row['data_particao'],row['horario']),axis=1)
chuva['id_estacao']=chuva['id_estacao'].astype(float)
chuva['id_estacao']=chuva['id_estacao'].astype(str)
chuva = chuva.sort_values(by=['DATE','id_estacao'])

<ipython-input-8-46736022e0e7>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  chuva['DATE'] = chuva.apply(lambda row: pd.datetime.combine(row['data_particao'],row['horario']),axis=1)


In [11]:
for elem in range(0,len(all_files)):
    dfbus = pd.read_parquet('/home/vinicius.vancellote/data/parquet/' + all_files[elem],engine='fastparquet')
    dfbus['LAT'] = dfbus['LAT'].astype(str)
    dfbus['LONG'] = dfbus['LONG'].astype(str)
    dfbus = dfbus.assign(geometry = ('POINT Z (' + dfbus['LONG'] + ' ' + dfbus['LAT'] + ' ' + '0.00000)'))
    cp_union = gpd.GeoDataFrame(
    dfbus.loc[:, [c for c in dfbus.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(dfbus["geometry"]),
    crs="epsg:4326",
    )
    dfjoin = gpd.sjoin(cp_union,zonasgpd,how='left')
    dfjoin['DATE'] = pd.to_datetime(dfjoin['DATE'],format = '%m-%d-%Y %H:%M:%S')
    dfjoin['OBJECTID_right']=dfjoin['OBJECTID_right'].astype(str)
    dfjoin = dfjoin.sort_values(by=['DATE','OBJECTID_right'])
    final_df = pd.merge_asof(dfjoin,chuva,on=['DATE'],left_by = ['OBJECTID_right'],right_by = ['id_estacao'],direction = 'forward')
    df_result = pd.DataFrame(final_df.drop(columns='geometry'))
    df_result['OBJECTID_right']=dfjoin['OBJECTID_right'].astype(str)
    df_result['INTERVAL'] = df_result['INTERVAL'].astype(str)
    df_result['horario'] = df_result['horario'].astype(str)
    df_result.to_parquet('/home/vinicius.vancellote/data/parquet/join/' + all_files[elem],engine='fastparquet')
    print(all_files[elem])

G1-2019-08-27.parquet
G1-2019-12-26.parquet
G1-2019-12-21.parquet
G1-2019-12-02.parquet
G1-2019-08-12.parquet
G1-2019-08-08.parquet
G1-2019-08-18.parquet
G1-2019-08-31.parquet
G1-2019-12-04.parquet
G1-2019-08-10.parquet
G1-2019-12-20.parquet
G1-2019-08-01.parquet
G1-2019-12-10.parquet
G1-2019-12-28.parquet
G1-2019-12-19.parquet
G1-2019-12-03.parquet
G1-2019-12-09.parquet
G1-2019-12-08.parquet
G1-2019-08-03.parquet
G1-2019-08-23.parquet
G1-2019-08-06.parquet
G1-2019-12-15.parquet
G1-2019-12-11.parquet
G1-2019-12-22.parquet
G1-2019-08-28.parquet
G1-2019-08-11.parquet
G1-2019-08-09.parquet
G1-2019-12-16.parquet
G1-2019-08-20.parquet
G1-2019-08-05.parquet
G1-2019-12-24.parquet
G1-2019-08-25.parquet
G1-2019-12-25.parquet
G1-2019-08-26.parquet
G1-2019-12-13.parquet
G1-2019-08-22.parquet
G1-2019-08-07.parquet
G1-2019-08-16.parquet
G1-2019-08-30.parquet
G1-2019-08-15.parquet
G1-2019-12-31.parquet
G1-2019-08-19.parquet
G1-2019-12-14.parquet
G1-2019-12-30.parquet
G1-2019-12-06.parquet
G1-2019-08

In [ ]:
chuva.dtypes

In [ ]:
df_result.dtypes

In [ ]:
print(df_result[160:164])